Drivers do selenium

Link para instalação do geckodriver (firefox)  
https://github.com/mozilla/geckodriver/releases

Link para instalação do chromedriver (Google Chrome)  
https://chromedriver.chromium.org/downloads

Após o download, devemos colocar o executável na mesma pasta que o python utilizado

# Constantes e imports

In [ ]:
#####################
#####CONSTANTES######
#####################
#dados do zoom
LINK = 'https://us02web.zoom.us/rec/share/i2ofasdasdUoLa4QT_S7yisumNksaddasumssEY70aJadssadsad00'
SENHA = 'ldwq4$e1'
#Browser:
#0 - Chrome
#1 - Firefox
BROWSER = 0

#A constante abaixo é para facilitar a execução do código
#Utilizando Cells - Run ALL
#True - Para a lista de participantes da sessão
#False - Para fazer o download do vídeo
LISTA_OU_DOWNLOAD = False

#####################
#######IMPORTS#######
#####################
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
from bs4 import BeautifulSoup as bs
import requests as r
import time

# Declaração de funções

## escolhe_browser

Escolhe o driver apropriado e maximiza a janela

In [ ]:
def escolhe_browser(browser):
    if browser:
        driver = webdriver.Firefox()
    else:
        driver = webdriver.Chrome('chromedriver')

    #maximiza a janela para garantir a exibição do chat
    driver.maximize_window()
    return driver

## login

Utiliza o link e a senha para fazer login

In [ ]:
def login(link,senha,driver):
    
    driver.get(link)
    
    action = ActionChains(driver)

    #localiza o campo de senha
    campo_de_senha = driver.find_element_by_xpath('/html/body/div[1]/div[3]/div[2]/div[2]/div/form/div/div/input[1]')
    
    #coloca a senha e aperta enter
    campo_de_senha.send_keys(senha)
    campo_de_senha.send_keys(Keys.ENTER)

## lista_participantes

Cria uma lista de participantes do chat

In [ ]:
def lista_participantes(driver):
    #espera o chat carregar
    delay = 20 # seconds
    try:
        _ = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.CLASS_NAME, 'user-name')))
    except:
        print ("Loading took too much time!")

    #acessa a página
    pagina = driver.page_source
    parsed = bs(pagina)

    #busca todos os usuários que participaram do chat
    tags_users = parsed.find_all("span", attrs={"class": "user-name"})

    #salva todos os usuários do chat em um set (sem repetição)
    participantes = set()
    for tag in tags_users:
        participantes.add(tag.text)

    #transforma o set em lista e ordena
    participantes = sorted(participantes)

    return participantes

## cria_link

Cria um link para download do vídeo

In [ ]:
def cria_link(driver):
    #Verifica se o chat carregou
    delay = 20 # seconds
    try:
        _ = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.CLASS_NAME, 'user-name')))
    except:
        print ("Loading took too much time!")

    #salva o código fonte da pagina
    pagina = driver.page_source
    parsed = bs(pagina)

    #procura pelo link do video, no player
    tag_link = parsed.find("div", attrs={"class": "player-view"})
    
    #separa o link por regex
    link = re.search('src=\"(http.+?)\" ',str(tag_link)).group(1)

    #altera o html fonte da página atraves de JS
    #substitui o título do chat por um link de download
    driver.execute_script("""document.querySelector("h2[class='title']").innerHTML="<a href="""+link+""">Pimba!</a>";""")
    
    action = ActionChains(driver)
    pimba = driver.find_element_by_xpath('/html/body/div/section/div/div[3]/div[2]/div/div/h2')
    
    #depois desse comando o usuário precisa clicar em salvar como, selecionar a pasta e escolher o nome do arquivo
    action.context_click(pimba).perform()

# Chamada de funções

## Para imprimir a lista de presença

In [ ]:
if LISTA_OU_DOWNLOAD:
    driver = escolhe_browser(BROWSER)

    login(LINK,SENHA,driver)

    participantes = lista_participantes(driver)

    driver.close()

    #Imprime a lista de participantes
    for participante in participantes: print(participante)

## Para fazer download do video

In [ ]:
if not LISTA_OU_DOWNLOAD:
    driver = escolhe_browser(BROWSER)

    login(LINK,SENHA,driver)

    cria_link(driver)

### Fecha o navegador

In [ ]:
#feche após salvar o link
driver.close()